In [1]:
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date
from env_vars import sentinel_username,sentinel_password
import glob
import pandas as pd
import subprocess

ModuleNotFoundError: No module named 'env_vars'

In [2]:
def get_api():
    
    return SentinelAPI(sentinel_username, sentinel_password, "https://scihub.copernicus.eu/apihub/")


def get_products_df(api, footprint, date_start, date_end,
                    area, raw, cloudcover,
                    platform='Sentinel-2'):
    
    products = api.query(footprint,
                         date=(date_start, date_end),
                         area_relation=area,
                         raw=raw,
                         platformname=platform,
                         cloudcoverpercentage=cloudcover)
    
    return api.to_dataframe(products)


def get_products_df_for_year(api, footprint, year, cloudcover, raw='L2A'):
    months_dict = {
        1: 31,
        2: 28,
        3: 31,
        4: 30,
        5: 31,
        6: 30,
        7: 31,
        8: 31,
        9: 30,
        10: 31,
        11: 30,
        12: 31
    }
    
    month_start = 1
    month_end = 13
    if year < 2015 or year > 2020:
        return None
    elif year == 2015:
        month_start = 7
    elif year == 2020:
        months_dict[2] = 29
        month_end = 8

    products_df = pd.DataFrame()
    for month in range(month_start, month_end):
        print('getting month', month)
        date_start = date(year, month, 1)
        date_end = date(year, month, months_dict[month])
        products_df_2 = get_products_df(api, footprint, date_start, date_end, area='IsWithin', raw=raw, cloudcover=cloudcover)
        products_df = pd.concat([products_df, products_df_2])
        products_df_3 = get_products_df(api, footprint, date_start, date_end, area='Intersects', raw=raw, cloudcover=cloudcover)
        products_df = pd.concat([products_df, products_df_2])
        print('Products so far:', len(products_df))
        
    return products_df


def find_ee_index_matches(api, products_df, ee_index):
    
    if len(products_df) == 0:
        print('No search results')
        return []
   
    products_df = products_df.reset_index()
    products_df = products_df.rename(columns={'index': 'sentinel_id'})
    products_df = products_df.drop_duplicates(subset=['tileid'])
    
    ee_index_2 = ee_index.reset_index()
    merged = products_df.merge(ee_index_2, left_on='title', right_on='PRODUCT_ID')
    rows = merged['index'].tolist()
    
    print(len(rows), 'total rows')
            
    return rows


def generate_tci_uri(ee_index, row):
    uri = ee_index.loc[row, 'BASE_URL']
    uri += '/GRANULE/'
    granule_id = ee_index.loc[row, 'GRANULE_ID']
    uri += granule_id
    uri += '/IMG_DATA/'
    tile_id = granule_id.split('_')[1]
    date = ee_index.loc[row, 'DATATAKE_IDENTIFIER'].split('_')[1]
    uri += f'{tile_id}_{date}_TCI.jp2'
    
    return uri


def download_tcis(ee_index, rows, dest_folder):
    
    cloud_env = r"C:\Users\David\AppData\Local\Google\Cloud SDK\cloud_env.bat"
    
    for row in rows:
        uri = generate_tci_uri(ee_index, row)
        subprocess.run([cloud_env, '&&', 'gsutil', 'cp', uri, dest_folder])
        
        
def get_tcis_for_year(year, footprint, ee_index, dest_folder, cloudcover=(1,5)):
    
    api = get_api()
    
    print('getting products')
    if year < 2015:
        print('ERROR: Year must be 2014 or above')
        return None
    else:
        products_df = get_products_df_for_year(api, footprint, year, cloudcover)
    print('finding rows')
    rows = find_ee_index_matches(api, products_df, ee_index)
    print('downloading tcis')
    download_tcis(ee_index, rows, dest_folder)

In [3]:
footprint = geojson_to_wkt(read_geojson('./data/Geometry/congo_basin_boundary/congo_basin_boundary_custom_v3.geojson'))

In [8]:
ee_index = pd.read_csv('earth-engine-index.csv')

KeyboardInterrupt: 

In [6]:
api = get_api()
products_df = get_products_df(api, footprint, date(2015, 7, 1), date(2015, 7, 31))
products_df.head()

,title,link,link_alternative,link_icon,summary,datatakesensingstart,beginposition,endposition,ingestiondate,orbitnumber,...,platformname,size,tileid,hv_order_tileid,filename,identifier,uuid,level1cpdiidentifier,granuleidentifier,datastripidentifier
55797ce2-d2a1-4e15-826c-75a4f421a21a,S2A_MSIL1C_20150708T081016_N0204_R078_T35NPA_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2015-07-08T08:10:16.027Z, Instrument: MS...",2015-07-08 08:10:16.027,2015-07-08 08:10:16.027,2015-07-08 08:10:16.027,2019-02-04 20:23:28.140,218,...,Sentinel-2,152.40 MB,35NPA,NA35P,S2A_MSIL1C_20150708T081016_N0204_R078_T35NPA_2...,S2A_MSIL1C_20150708T081016_N0204_R078_T35NPA_2...,55797ce2-d2a1-4e15-826c-75a4f421a21a,S2A_OPER_MSI_L1C_TL_EPA__20160807T083950_A0002...,S2A_OPER_MSI_L1C_TL_EPA__20160807T083950_A0002...,S2A_OPER_MSI_L1C_DS_EPA__20160807T083950_S2015...
1bce998b-c4f2-42d3-b2a1-de8db9187439,S2A_MSIL1C_20150708T081016_N0204_R078_T35MPV_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2015-07-08T08:10:16.027Z, Instrument: MS...",2015-07-08 08:10:16.027,2015-07-08 08:10:16.027,2015-07-08 08:10:16.027,2019-02-04 20:16:31.232,218,...,Sentinel-2,307.36 MB,35MPV,MV35P,S2A_MSIL1C_20150708T081016_N0204_R078_T35MPV_2...,S2A_MSIL1C_20150708T081016_N0204_R078_T35MPV_2...,1bce998b-c4f2-42d3-b2a1-de8db9187439,S2A_OPER_MSI_L1C_TL_EPA__20160807T083950_A0002...,S2A_OPER_MSI_L1C_TL_EPA__20160807T083950_A0002...,S2A_OPER_MSI_L1C_DS_EPA__20160807T083950_S2015...
0ef81aa0-0f21-4f3d-a04e-a03b44f67370,S2A_MSIL1C_20150727T085946_N0204_R064_T34MGE_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2015-07-27T08:59:46.027Z, Instrument: MS...",2015-07-27 08:59:46.027,2015-07-27 08:59:46.027,2015-07-27 08:59:46.027,2019-02-04 18:15:46.498,490,...,Sentinel-2,612.50 MB,34MGE,ME34G,S2A_MSIL1C_20150727T085946_N0204_R064_T34MGE_2...,S2A_MSIL1C_20150727T085946_N0204_R064_T34MGE_2...,0ef81aa0-0f21-4f3d-a04e-a03b44f67370,S2A_OPER_MSI_L1C_TL_EPA__20160817T102829_A0004...,S2A_OPER_MSI_L1C_TL_EPA__20160817T102829_A0004...,S2A_OPER_MSI_L1C_DS_EPA__20160817T102829_S2015...
37762ba4-ae91-4eb6-b1e2-003111c39129,S2A_MSIL1C_20150727T085946_N0204_R064_T34NGH_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2015-07-27T08:59:46.027Z, Instrument: MS...",2015-07-27 08:59:46.027,2015-07-27 08:59:46.027,2015-07-27 08:59:46.027,2019-02-04 17:52:21.908,490,...,Sentinel-2,238.87 MB,34NGH,NH34G,S2A_MSIL1C_20150727T085946_N0204_R064_T34NGH_2...,S2A_MSIL1C_20150727T085946_N0204_R064_T34NGH_2...,37762ba4-ae91-4eb6-b1e2-003111c39129,S2A_OPER_MSI_L1C_TL_EPA__20160817T102829_A0004...,S2A_OPER_MSI_L1C_TL_EPA__20160817T102829_A0004...,S2A_OPER_MSI_L1C_DS_EPA__20160817T102829_S2015...
bb4e20f3-5e90-4fbc-9e38-6dcb20489557,S2A_MSIL1C_20150727T085946_N0204_R064_T34NGG_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2015-07-27T08:59:46.027Z, Instrument: MS...",2015-07-27 08:59:46.027,2015-07-27 08:59:46.027,2015-07-27 08:59:46.027,2019-02-04 17:52:00.960,490,...,Sentinel-2,681.42 MB,34NGG,NG34G,S2A_MSIL1C_20150727T085946_N0204_R064_T34NGG_2...,S2A_MSIL1C_20150727T085946_N0204_R064_T34NGG_2...,bb4e20f3-5e90-4fbc-9e38-6dcb20489557,S2A_OPER_MSI_L1C_TL_EPA__20160817T102829_A0004...,S2A_OPER_MSI_L1C_TL_EPA__20160817T102829_A0004...,S2A_OPER_MSI_L1C_DS_EPA__20160817T102829_S2015...


In [7]:
products_df.columns

Index(['title', 'link', 'link_alternative', 'link_icon', 'summary',
       'datatakesensingstart', 'beginposition', 'endposition', 'ingestiondate',
       'orbitnumber', 'relativeorbitnumber', 'cloudcoverpercentage',
       'gmlfootprint', 'format', 'instrumentshortname',
       'sensoroperationalmode', 'instrumentname', 'footprint', 's2datatakeid',
       'platformidentifier', 'orbitdirection', 'platformserialidentifier',
       'processingbaseline', 'processinglevel', 'producttype', 'platformname',
       'size', 'tileid', 'hv_order_tileid', 'filename', 'identifier', 'uuid',
       'level1cpdiidentifier', 'granuleidentifier', 'datastripidentifier'],
      dtype='object')

In [9]:
products_df[products_df['title'].str.contains('L2A')]

,title,link,link_alternative,link_icon,summary,datatakesensingstart,beginposition,endposition,ingestiondate,orbitnumber,...,platformname,size,tileid,hv_order_tileid,filename,identifier,uuid,level1cpdiidentifier,granuleidentifier,datastripidentifier


In [13]:
ee_index_2 = ee_index.reset_index()

products_df = products_df.rename(columns={'index': 'sentinel_id'})
merged = products_df.merge(ee_index_2, left_on='title', right_on='PRODUCT_ID')
merged.head()

,sentinel_id,title,link,link_alternative,link_icon,summary,datatakesensingstart,beginposition,endposition,ingestiondate,...,SENSING_TIME,TOTAL_SIZE,CLOUD_COVER,GEOMETRIC_QUALITY_FLAG,GENERATION_TIME,NORTH_LAT,SOUTH_LAT,WEST_LON,EAST_LON,BASE_URL
0,55797ce2-d2a1-4e15-826c-75a4f421a21a,S2A_MSIL1C_20150708T081016_N0204_R078_T35NPA_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2015-07-08T08:10:16.027Z, Instrument: MS...",2015-07-08 08:10:16.027,2015-07-08 08:10:16.027,2015-07-08 08:10:16.027,2019-02-04 20:23:28.140,...,2015-07-08T08:21:58.000000Z,159641454.0,1.3745,PASSED,2015-07-08T08:21:58.000000Z,0.904542,-0.088357,28.602039,28.885221,gs://gcp-public-data-sentinel-2/tiles/35/N/PA/...
1,1bce998b-c4f2-42d3-b2a1-de8db9187439,S2A_MSIL1C_20150708T081016_N0204_R078_T35MPV_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2015-07-08T08:10:16.027Z, Instrument: MS...",2015-07-08 08:10:16.027,2015-07-08 08:10:16.027,2015-07-08 08:10:16.027,2019-02-04 20:16:31.232,...,2015-07-08T08:21:58.000000Z,322125342.0,2.9824,PASSED,2015-07-08T08:21:58.000000Z,0.000090,-0.993003,28.402809,28.885269,gs://gcp-public-data-sentinel-2/tiles/35/M/PV/...
2,d3af0e3f-24ee-4104-b5b7-7f90e69781b7,S2A_MSIL1C_20150708T081016_N0204_R078_T35MPU_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2015-07-08T08:10:16.027Z, Instrument: MS...",2015-07-08 08:10:16.027,2015-07-08 08:10:16.027,2015-07-08 08:10:16.027,2019-02-04 20:02:59.677,...,2015-07-08T08:21:58.000000Z,492387691.0,5.0505,PASSED,2015-07-08T08:21:58.000000Z,-0.903786,-1.897246,28.202183,28.886014,gs://gcp-public-data-sentinel-2/tiles/35/M/PU/...
3,6ba536c1-41ad-4c15-90ca-6e78c3fe10b7,S2A_MSIL1C_20150708T081016_N0204_R078_T35NQB_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2015-07-08T08:10:16.027Z, Instrument: MS...",2015-07-08 08:10:16.027,2015-07-08 08:10:16.027,2015-07-08 08:10:16.027,2019-02-04 19:58:05.495,...,2015-07-08T08:21:58.000000Z,722956185.0,0.6652,PASSED,2015-07-08T08:21:58.000000Z,1.808778,0.815551,28.800827,29.783921,gs://gcp-public-data-sentinel-2/tiles/35/N/QB/...
4,d89f7c7b-7e9e-4df2-a946-b6804920ba69,S2A_MSIL1C_20150727T085946_N0204_R064_T34NHF_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2015-07-27T08:59:46.027Z, Instrument: MS...",2015-07-27 08:59:46.027,2015-07-27 08:59:46.027,2015-07-27 08:59:46.027,2019-02-04 18:18:31.878,...,2015-07-27T08:59:42.000000Z,135849421.0,0.0000,PASSED,2015-07-27T08:59:42.000000Z,0.903994,-0.088293,23.694757,23.989523,gs://gcp-public-data-sentinel-2/tiles/34/N/HF/...


In [14]:
merged.columns

Index(['sentinel_id', 'title', 'link', 'link_alternative', 'link_icon',
       'summary', 'datatakesensingstart', 'beginposition', 'endposition',
       'ingestiondate', 'orbitnumber', 'relativeorbitnumber',
       'cloudcoverpercentage', 'gmlfootprint', 'format', 'instrumentshortname',
       'sensoroperationalmode', 'instrumentname', 'footprint', 's2datatakeid',
       'platformidentifier', 'orbitdirection', 'platformserialidentifier',
       'processingbaseline', 'processinglevel', 'producttype', 'platformname',
       'size', 'tileid', 'hv_order_tileid', 'filename', 'identifier', 'uuid',
       'level1cpdiidentifier', 'granuleidentifier', 'datastripidentifier',
       'index', 'GRANULE_ID', 'PRODUCT_ID', 'DATATAKE_IDENTIFIER', 'MGRS_TILE',
       'SENSING_TIME', 'TOTAL_SIZE', 'CLOUD_COVER', 'GEOMETRIC_QUALITY_FLAG',
       'GENERATION_TIME', 'NORTH_LAT', 'SOUTH_LAT', 'WEST_LON', 'EAST_LON',
       'BASE_URL'],
      dtype='object')

In [15]:
merged['index'].tolist()

[7725269,
 11241382,
 11273022,
 8764011,
 8231314,
 822721,
 3701925,
 1911926,
 5710807,
 8508100,
 12158332,
 2279478,
 805527,
 13025774,
 4054446,
 11016874,
 9316782,
 7790633,
 8452064,
 8166815,
 1410603,
 1483886,
 4477872,
 4543905,
 12628861]

In [20]:
rows_array = []

d = {}

for month in ['Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']:
    print(f'Starting month {month}')
    month_num = months_dict[month][0]
    last_day = months_dict[month][1]
    
    api = get_api()
    products = get_products(api, footprint, date(2015, month_num, 1), date(2015, month_num, last_day))
    products_df = api.to_dataframe(products)
    products_df = products_df.reset_index()
    products_df = products_df.rename(columns={'index': 'sentinel_id'})
    products_df = products_df.drop_duplicates(subset=['tileid'])

    merged = products_df.merge(ee_index, left_on='title', right_on='PRODUCT_ID')
    rows = merged['index'].tolist()
    rows_array.append(rows)
    d[month] = len(rows)
    print(f'Ending month {month}')

Starting month Jul
Ending month Jul
Starting month Aug
Ending month Aug
Starting month Sep
Ending month Sep
Starting month Oct
Ending month Oct
Starting month Nov
Ending month Nov
Starting month Dec


Querying products: 100%|██████████████████████████████████████████████████████| 248/248 [00:07<00:00, 31.23 products/s]


Ending month Dec


In [21]:
d

{'Jul': 25, 'Aug': 26, 'Sep': 47, 'Oct': 8, 'Nov': 31, 'Dec': 126}

In [22]:
api = get_api()
products = get_products(api, footprint, date(2015, 7, 1), date(2015, 12, 31))
products_df = api.to_dataframe(products)
products_df = products_df.reset_index()
products_df = products_df.rename(columns={'index': 'sentinel_id'})
products_df = products_df.drop_duplicates(subset=['tileid'])

merged = products_df.merge(ee_index, left_on='title', right_on='PRODUCT_ID')
rows = merged['index'].tolist()

len(rows)

Querying products: 100%|██████████████████████████████████████████████████████| 418/418 [00:33<00:00, 12.66 products/s]


158

In [23]:
api = get_api()
products = get_products(api, footprint, date(2016, 1, 1), date(2016, 12, 31))
products_df = api.to_dataframe(products)
products_df = products_df.reset_index()
products_df = products_df.rename(columns={'index': 'sentinel_id'})
products_df = products_df.drop_duplicates(subset=['tileid'])

merged = products_df.merge(ee_index, left_on='title', right_on='PRODUCT_ID')
rows = merged['index'].tolist()

len(rows)

Querying products: 100%|████████████████████████████████████████████████████| 1864/1864 [05:04<00:00,  6.13 products/s]


187

In [11]:
get_tcis_for_year(2016, footprint, ee_index)

getting products


Querying products: 100%|████████████████████████████████████████████████████| 1864/1864 [04:46<00:00,  6.50 products/s]


finding rows
downloading tcis


In [7]:
get_tcis_for_year(2015, footprint, ee_index)

getting products


Querying products: 100%|██████████████████████████████████████████████████████| 418/418 [00:34<00:00, 12.26 products/s]


finding rows
downloading tcis


In [10]:
get_tcis_for_year(2017, footprint, ee_index)

getting products


Querying products: 100%|████████████████████████████████████████████████████| 2351/2351 [09:38<00:00,  4.07 products/s]


finding rows
downloading tcis


In [20]:
get_tcis_for_year(2018, footprint, ee_index)

getting products
getting month 1





Querying products:  16%|█████████▉                                                    | 100/622 [00:00<?, ? products/s]


Querying products:  32%|█████████████████▎                                    | 200/622 [00:05<00:24, 16.97 products/s]


Querying products:  48%|██████████████████████████                            | 300/622 [00:13<00:20, 15.76 products/s]


Querying products:  64%|██████████████████████████████████▋                   | 400/622 [00:19<00:14, 15.68 products/s]


Querying products:  80%|███████████████████████████████████████████▍          | 500/622 [00:26<00:07, 15.61 products/s]


Querying products:  96%|████████████████████████████████████████████████████  | 600/622 [00:31<00:01, 16.12 products/s]


Querying products: 100%|██████████████████████████████████████████████████████| 622/622 [00:38<00:00, 16.24 products/s]


getting month 2





Querying products:  40%|████████████████████████▊                                     | 100/250 [00:00<?, ? products/s]


Querying products:  80%|███████████████████████████████████████████▏          | 200/250 [00:05<00:02, 17.01 products/s]


Querying products: 100%|██████████████████████████████████████████████████████| 250/250 [00:10<00:00, 22.81 products/s]


getting month 3


...

getting month 11





Querying products:  35%|█████████████████████▊                                        | 100/284 [00:00<?, ? products/s]


Querying products:  70%|██████████████████████████████████████                | 200/284 [00:07<00:06, 12.67 products/s]


Querying products: 100%|██████████████████████████████████████████████████████| 284/284 [00:15<00:00, 18.34 products/s]


getting month 12





Querying products:  32%|███████████████████▊                                          | 100/312 [00:00<?, ? products/s]


Querying products:  64%|██████████████████████████████████▌                   | 200/312 [00:06<00:07, 14.29 products/s]


Querying products:  96%|███████████████████████████████████████████████████▉  | 300/312 [00:13<00:00, 14.36 products/s]


Querying products: 100%|██████████████████████████████████████████████████████| 312/312 [00:20<00:00, 15.59 products/s]


finding rows
downloading tcis


In [5]:
months_dict = {
    1: 31,
    2: 28,
    3: 31,
    4: 30,
    5: 31,
    6: 30,
    7: 31,
    8: 31,
    9: 30,
    10: 31,
    11: 30,
    12: 31
}

api = get_api()
year = 2019
products_df = pd.DataFrame()
for month in range(7, 13):
    print('getting month', month)
    date_start = date(year, month, 1)
    date_end = date(year, month, months_dict[month])
    products_df_2 = get_products_df(api, footprint, date_start, date_end)
    products_df = pd.concat([products_df, products_df_2])
    
products_df = products_df.reset_index()
products_df = products_df.rename(columns={'index': 'sentinel_id'})
products_df = products_df.drop_duplicates(subset=['tileid'])

getting month 7


Querying products: 100%|██████████████████████████████████████████████████████| 305/305 [00:50<00:00,  6.06 products/s]


getting month 8


Querying products: 100%|██████████████████████████████████████████████████████| 288/288 [00:21<00:00, 13.40 products/s]


getting month 9


Querying products: 100%|██████████████████████████████████████████████████████| 199/199 [00:09<00:00, 21.27 products/s]


getting month 10
getting month 11


Querying products: 100%|██████████████████████████████████████████████████████| 147/147 [00:07<00:00, 19.72 products/s]


getting month 12


Querying products: 100%|██████████████████████████████████████████████████████| 345/345 [00:16<00:00, 20.31 products/s]


In [6]:
ee_index_2 = ee_index.reset_index()
merged = products_df.merge(ee_index_2, left_on='title', right_on='PRODUCT_ID')
print(merged.shape)

(195, 51)


In [7]:
print(products_df.shape)

(195, 36)


In [5]:
get_tcis_for_year(2019, footprint, ee_index)

getting products
getting month 1


Querying products: 100%|██████████████████████████████████████████████████████| 318/318 [00:28<00:00, 11.23 products/s]


getting month 2


Querying products: 100%|██████████████████████████████████████████████████████| 311/311 [00:24<00:00, 12.58 products/s]


getting month 3


...

getting month 10
getting month 11


Querying products: 100%|██████████████████████████████████████████████████████| 147/147 [00:08<00:00, 17.22 products/s]


getting month 12


Querying products: 100%|██████████████████████████████████████████████████████| 345/345 [00:18<00:00, 18.61 products/s]


finding rows
downloading tcis


In [9]:
get_tcis_for_year(2020, footprint, ee_index)

getting products
getting month 1


Querying products: 100%|██████████████████████████████████████████████████████| 470/470 [00:53<00:00,  8.82 products/s]


getting month 2


Querying products: 100%|██████████████████████████████████████████████████████| 467/467 [00:47<00:00,  9.91 products/s]


getting month 3


...

getting month 6


Querying products: 100%|██████████████████████████████████████████████████████| 343/343 [00:32<00:00, 10.63 products/s]


getting month 7


Querying products: 100%|██████████████████████████████████████████████████████| 219/219 [00:12<00:00, 18.18 products/s]


finding rows
downloading tcis


In [4]:
api = get_api()

help(api.query)

Help on method query in module sentinelsat.sentinel:

query(area=None, date=None, raw=None, area_relation='Intersects', order_by=None, limit=None, offset=0, **keywords) method of sentinelsat.sentinel.SentinelAPI instance
    Query the OpenSearch API with the coordinates of an area, a date interval
    and any other search keywords accepted by the API.
    
    Parameters
    ----------
    area : str, optional
        The area of interest formatted as a Well-Known Text string.
    date : tuple of (str or datetime) or str, optional
        A time interval filter based on the Sensing Start Time of the products.
        Expects a tuple of (start, end), e.g. ("NOW-1DAY", "NOW").
        The timestamps can be either a Python datetime or a string in one of the
        following formats:
    
            - yyyyMMdd
            - yyyy-MM-ddThh:mm:ss.SSSZ (ISO-8601)
            - yyyy-MM-ddThh:mm:ssZ
            - NOW
            - NOW-<n>DAY(S) (or HOUR(S), MONTH(S), etc.)
            - NOW+<n

In [6]:
footprint = geojson_to_wkt(read_geojson('./data/Geometry/republic_of_the_congo_boundary.geojson'))

ee_index = pd.read_csv('earth-engine-index.csv')

In [8]:
get_tcis_for_year(2015, footprint, ee_index, 'D:/canopy_data/republic-of-the-congo/2015')

getting products
getting month 7
getting month 8
getting month 9
getting month 10
getting month 11
getting month 12
finding rows


KeyError: 'title'

In [10]:
get_tcis_for_year(2015, footprint, ee_index, 'D:/canopy_data/republic-of-the-congo/2015', cloudcover=(0,10))

getting products
getting month 7
getting month 8
getting month 9
getting month 10
getting month 11
getting month 12
finding rows


KeyError: 'title'

In [13]:
get_tcis_for_year(2020, footprint, ee_index, 'D:/canopy_data/republic-of-the-congo/2015')

getting products
getting month 1
getting month 2
getting month 3
getting month 4
getting month 5
getting month 6
getting month 7
finding rows
No search results
downloading tcis


In [14]:
footprint

'MULTILINESTRING((14.4291 2.1559,14.4337 2.1559,14.4401 2.1594,14.4456 2.1594,14.4908 2.1774,14.5253 2.1880,14.5290 2.1900,14.5388 2.1954,14.5459 2.1970,14.5570 2.2115,14.5674 2.2036,14.5733 2.2029,14.5809 2.2111,14.5832 2.2118,14.5866 2.2104,14.5912 2.2017,14.5939 2.2007,14.6008 2.1984,14.6067 2.1968,14.6122 2.1908,14.6163 2.1807,14.6198 2.1746,14.6306 2.1563,14.6425 2.1551,14.6478 2.1512,14.6494 2.1468,14.6457 2.1378,14.6509 2.1291,14.6560 2.1275,14.6800 2.1300,14.6859 2.1306,14.6905 2.1285,14.6989 2.1197,14.7026 2.1092,14.7149 2.1082,14.7231 2.1038,14.7268 2.1038,14.7270 2.1080,14.7223 2.1151,14.7225 2.1190,14.7241 2.1220,14.7296 2.1222,14.7554 2.1035,14.7568 2.1007,14.7554 2.0950,14.7437 2.0953,14.7423 2.0911,14.7444 2.0874,14.7499 2.0867,14.7613 2.0947,14.7721 2.0954,14.7753 2.0926,14.7773 2.0862,14.7766 2.0807,14.7702 2.0683,14.7708 2.0657,14.7750 2.0616,14.7800 2.0611,14.7942 2.0719,14.7976 2.0717,14.8104 2.0581,14.8147 2.0576,14.8276 2.0624,14.8358 2.0704,14.8367 2.0719,14.8411

In [15]:
footprint = geojson_to_wkt(read_geojson('./data/Geometry/republic_of_the_congo_boundary_simplified.geojson'))

In [16]:
get_tcis_for_year(2020, footprint, ee_index, 'D:/canopy_data/republic-of-the-congo/2020')

getting products
getting month 1
getting month 2
getting month 3
getting month 4
getting month 5
getting month 6
getting month 7
finding rows
No search results
downloading tcis


In [21]:
get_tcis_for_year(2020, footprint, ee_index, 'D:/canopy_data/republic-of-the-congo/2020')

getting products
getting month 1
Products so far: 0
getting month 2
Products so far: 0
getting month 3
Products so far: 0
getting month 4
Products so far: 0
getting month 5
Products so far: 0
getting month 6
Products so far: 0
getting month 7
Products so far: 0
finding rows
No search results
downloading tcis


In [12]:
footprint = geojson_to_wkt(read_geojson('./data/Geometry/republic_of_the_congo_boundary_bounding_box.geojson'))

In [5]:
footprint

'POLYGON((11.6396 -4.8206,18.4909 -4.8206,18.4909 3.6517,11.6396 3.6517,11.6396 -4.8206))'

In [14]:
get_tcis_for_year(2020, footprint, ee_index, 'D:/canopy_data/republic-of-the-congo/2020')

getting products
getting month 1
Products so far: 56
getting month 2
Products so far: 114
getting month 3
Products so far: 144
getting month 4
Products so far: 198
getting month 5
Products so far: 260
getting month 6
Products so far: 296
getting month 7
Products so far: 346
finding rows
downloading tcis


In [13]:
api = get_api()

products_df = get_products_df_for_year(api, footprint, 2020, cloudcover=(1,5), raw='L2A')

products_df.shape

getting month 1
Products so far: 0
getting month 2
Products so far: 0
getting month 3
Products so far: 0
getting month 4
Products so far: 0
getting month 5
Products so far: 0
getting month 6
Products so far: 0
getting month 7
Products so far: 0


(0, 0)

In [14]:
products_df = get_products_df_for_year(api, footprint, 2020, cloudcover=(0,10), raw='L2A')

getting month 1
Products so far: 0
getting month 2
Products so far: 0
getting month 3
Products so far: 0
getting month 4
Products so far: 0
getting month 5
Products so far: 0
getting month 6
Products so far: 0
getting month 7
Products so far: 0


In [16]:
products_df = get_products_df_for_year(api, footprint, 2020, cloudcover=(1,5), raw='2A')
products_df.shape

getting month 1


KeyboardInterrupt: 

In [17]:
help(api.query)

Help on method query in module sentinelsat.sentinel:

query(area=None, date=None, raw=None, area_relation='Intersects', order_by=None, limit=None, offset=0, **keywords) method of sentinelsat.sentinel.SentinelAPI instance
    Query the OpenSearch API with the coordinates of an area, a date interval
    and any other search keywords accepted by the API.
    
    Parameters
    ----------
    area : str, optional
        The area of interest formatted as a Well-Known Text string.
    date : tuple of (str or datetime) or str, optional
        A time interval filter based on the Sensing Start Time of the products.
        Expects a tuple of (start, end), e.g. ("NOW-1DAY", "NOW").
        The timestamps can be either a Python datetime or a string in one of the
        following formats:
    
            - yyyyMMdd
            - yyyy-MM-ddThh:mm:ss.SSSZ (ISO-8601)
            - yyyy-MM-ddThh:mm:ssZ
            - NOW
            - NOW-<n>DAY(S) (or HOUR(S), MONTH(S), etc.)
            - NOW+<n

In [20]:
products_df = get_products_df(api, footprint, date(2020, 1, 1), date(2020, 1, 31),
                              area='IsWithin', raw='2A', cloudcover=(1,5),
                              platform='Sentinel-2')
                              
products_df.shape

(53, 40)

In [21]:
products_df

,title,link,link_alternative,link_icon,summary,beginposition,endposition,ingestiondate,orbitnumber,relativeorbitnumber,...,platformname,size,filename,level1cpdiidentifier,identifier,uuid,datatakesensingstart,sensoroperationalmode,tileid,hv_order_tileid
25593eba-df28-4e35-ad6a-e0c7743cd8be,S2A_MSIL2A_20200128T090221_N0213_R007_T33MYT_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2020-01-28T09:02:21.024Z, Instrument: MS...",2020-01-28 09:02:21.024,2020-01-28 09:02:21.024,2020-01-28 16:11:36.895,24028,7,...,Sentinel-2,1016.60 MB,S2A_MSIL2A_20200128T090221_N0213_R007_T33MYT_2...,S2A_OPER_MSI_L1C_TL_SGS__20200128T105959_A0240...,S2A_MSIL2A_20200128T090221_N0213_R007_T33MYT_2...,25593eba-df28-4e35-ad6a-e0c7743cd8be,NaT,NaN,NaN,NaN
2c8ac4fc-e6ea-4871-aca3-0b93a0e5e7a6,S2A_MSIL2A_20200128T090221_N0213_R007_T33MYU_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2020-01-28T09:02:21.024Z, Instrument: MS...",2020-01-28 09:02:21.024,2020-01-28 09:02:21.024,2020-01-28 16:09:01.560,24028,7,...,Sentinel-2,1.01 GB,S2A_MSIL2A_20200128T090221_N0213_R007_T33MYU_2...,S2A_OPER_MSI_L1C_TL_SGS__20200128T105959_A0240...,S2A_MSIL2A_20200128T090221_N0213_R007_T33MYU_2...,2c8ac4fc-e6ea-4871-aca3-0b93a0e5e7a6,NaT,NaN,NaN,NaN
f53e1428-fd5f-4afa-8645-dd239e6b1d70,S2A_MSIL2A_20200128T090221_N0213_R007_T33NYA_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2020-01-28T09:02:21.024Z, Instrument: MS...",2020-01-28 09:02:21.024,2020-01-28 09:02:21.024,2020-01-28 16:08:59.513,24028,7,...,Sentinel-2,1.00 GB,S2A_MSIL2A_20200128T090221_N0213_R007_T33NYA_2...,S2A_OPER_MSI_L1C_TL_SGS__20200128T105959_A0240...,S2A_MSIL2A_20200128T090221_N0213_R007_T33NYA_2...,f53e1428-fd5f-4afa-8645-dd239e6b1d70,NaT,NaN,NaN,NaN
5d840bb8-cd44-467a-b1cd-261f1d8b77f3,S2A_MSIL2A_20200128T090221_N0213_R007_T33NXB_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2020-01-28T09:02:21.024Z, Instrument: MS...",2020-01-28 09:02:21.024,2020-01-28 09:02:21.024,2020-01-28 16:08:14.995,24028,7,...,Sentinel-2,598.48 MB,S2A_MSIL2A_20200128T090221_N0213_R007_T33NXB_2...,S2A_OPER_MSI_L1C_TL_SGS__20200128T105959_A0240...,S2A_MSIL2A_20200128T090221_N0213_R007_T33NXB_2...,5d840bb8-cd44-467a-b1cd-261f1d8b77f3,NaT,NaN,NaN,NaN
be23b716-7c71-41d0-9246-cfdd378c9ecb,S2A_MSIL2A_20200128T090221_N0213_R007_T33NXC_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2020-01-28T09:02:21.024Z, Instrument: MS...",2020-01-28 09:02:21.024,2020-01-28 09:02:21.024,2020-01-28 16:07:44.881,24028,7,...,Sentinel-2,393.61 MB,S2A_MSIL2A_20200128T090221_N0213_R007_T33NXC_2...,S2A_OPER_MSI_L1C_TL_SGS__20200128T105959_A0240...,S2A_MSIL2A_20200128T090221_N0213_R007_T33NXC_2...,be23b716-7c71-41d0-9246-cfdd378c9ecb,NaT,NaN,NaN,NaN
2d0425a9-5973-4ff6-836d-0d09cc99441f,S2A_MSIL2A_20200128T090221_N0213_R007_T34MBC_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2020-01-28T09:02:21.024Z, Instrument: MS...",2020-01-28 09:02:21.024,2020-01-28 09:02:21.024,2020-01-28 16:06:29.147,24028,7,...,Sentinel-2,25.18 MB,S2A_MSIL2A_20200128T090221_N0213_R007_T34MBC_2...,S2A_OPER_MSI_L1C_TL_SGS__20200128T105959_A0240...,S2A_MSIL2A_20200128T090221_N0213_R007_T34MBC_2...,2d0425a9-5973-4ff6-836d-0d09cc99441f,NaT,NaN,NaN,NaN
4914140e-619b-459b-8dac-86cca42a0321,S2A_MSIL2A_20200128T090221_N0213_R007_T33MXU_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/api

In [22]:
products_df['title'][0]

'S2A_MSIL2A_20200128T090221_N0213_R007_T33MYT_20200128T113844'

In [23]:
products_df = get_products_df(api, footprint, date(2020, 1, 1), date(2020, 1, 31),
                              area='IsWithin', raw='L2A', cloudcover=(1,5),
                              platform='Sentinel-2')

products_df.shape

(0, 0)

In [ ]:
products_df = get_products_df(api, footprint, date(2020, 1, 1), date(2020, 1, 31),
                              area='IsWithin', raw='product', cloudcover=(1,5),
                              platform='Sentinel-2')

In [25]:
api.get_product_odata('be23b716-7c71-41d0-9246-cfdd378c9ecb', full=True)

{'id': 'be23b716-7c71-41d0-9246-cfdd378c9ecb',
 'title': 'S2A_MSIL2A_20200128T090221_N0213_R007_T33NXC_20200128T113844',
 'size': 412789165,
 'md5': 'D9911974ABDCCEE844ADB036037B80CC',
 'date': datetime.datetime(2020, 1, 28, 9, 2, 21, 24000),
 'footprint': 'POLYGON((16.42522235365677 1.72019435407441,16.446941572335422 1.818900558486982,16.47953217969016 1.967482097566204,16.512104888858914 2.115993634301595,16.54464340329782 2.264454880863955,16.577141921346062 2.412918747940549,16.609702864090963 2.561411227510657,16.64224814285553 2.709991649710312,16.642902013939256 2.712976760451385,16.887178547109823 2.712693903442947,16.885921359235475 1.719856058353082,16.42522235365677 1.72019435407441))',
 'url': "https://scihub.copernicus.eu/apihub/odata/v1/Products('be23b716-7c71-41d0-9246-cfdd378c9ecb')/$value",
 'Online': True,
 'Creation Date': datetime.datetime(2020, 1, 28, 16, 8, 3, 39000),
 'Ingestion Date': datetime.datetime(2020, 1, 28, 16, 7, 44, 881000),
 'Aot retrieval accuracy':

In [26]:
products_df = get_products_df(api, footprint, date(2020, 1, 1), date(2020, 1, 31),
                              area='IsWithin', raw='Level-2A', cloudcover=(1,5),
                              platform='Sentinel-2')

products_df.shape

(53, 40)

In [27]:
products_df.head()

,title,link,link_alternative,link_icon,summary,beginposition,endposition,ingestiondate,orbitnumber,relativeorbitnumber,...,platformname,size,filename,level1cpdiidentifier,identifier,uuid,datatakesensingstart,sensoroperationalmode,tileid,hv_order_tileid
25593eba-df28-4e35-ad6a-e0c7743cd8be,S2A_MSIL2A_20200128T090221_N0213_R007_T33MYT_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2020-01-28T09:02:21.024Z, Instrument: MS...",2020-01-28 09:02:21.024,2020-01-28 09:02:21.024,2020-01-28 16:11:36.895,24028,7,...,Sentinel-2,1016.60 MB,S2A_MSIL2A_20200128T090221_N0213_R007_T33MYT_2...,S2A_OPER_MSI_L1C_TL_SGS__20200128T105959_A0240...,S2A_MSIL2A_20200128T090221_N0213_R007_T33MYT_2...,25593eba-df28-4e35-ad6a-e0c7743cd8be,NaT,NaN,NaN,NaN
2c8ac4fc-e6ea-4871-aca3-0b93a0e5e7a6,S2A_MSIL2A_20200128T090221_N0213_R007_T33MYU_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2020-01-28T09:02:21.024Z, Instrument: MS...",2020-01-28 09:02:21.024,2020-01-28 09:02:21.024,2020-01-28 16:09:01.560,24028,7,...,Sentinel-2,1.01 GB,S2A_MSIL2A_20200128T090221_N0213_R007_T33MYU_2...,S2A_OPER_MSI_L1C_TL_SGS__20200128T105959_A0240...,S2A_MSIL2A_20200128T090221_N0213_R007_T33MYU_2...,2c8ac4fc-e6ea-4871-aca3-0b93a0e5e7a6,NaT,NaN,NaN,NaN
f53e1428-fd5f-4afa-8645-dd239e6b1d70,S2A_MSIL2A_20200128T090221_N0213_R007_T33NYA_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2020-01-28T09:02:21.024Z, Instrument: MS...",2020-01-28 09:02:21.024,2020-01-28 09:02:21.024,2020-01-28 16:08:59.513,24028,7,...,Sentinel-2,1.00 GB,S2A_MSIL2A_20200128T090221_N0213_R007_T33NYA_2...,S2A_OPER_MSI_L1C_TL_SGS__20200128T105959_A0240...,S2A_MSIL2A_20200128T090221_N0213_R007_T33NYA_2...,f53e1428-fd5f-4afa-8645-dd239e6b1d70,NaT,NaN,NaN,NaN
5d840bb8-cd44-467a-b1cd-261f1d8b77f3,S2A_MSIL2A_20200128T090221_N0213_R007_T33NXB_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2020-01-28T09:02:21.024Z, Instrument: MS...",2020-01-28 09:02:21.024,2020-01-28 09:02:21.024,2020-01-28 16:08:14.995,24028,7,...,Sentinel-2,598.48 MB,S2A_MSIL2A_20200128T090221_N0213_R007_T33NXB_2...,S2A_OPER_MSI_L1C_TL_SGS__20200128T105959_A0240...,S2A_MSIL2A_20200128T090221_N0213_R007_T33NXB_2...,5d840bb8-cd44-467a-b1cd-261f1d8b77f3,NaT,NaN,NaN,NaN
be23b716-7c71-41d0-9246-cfdd378c9ecb,S2A_MSIL2A_20200128T090221_N0213_R007_T33NXC_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2020-01-28T09:02:21.024Z, Instrument: MS...",2020-01-28 09:02:21.024,2020-01-28 09:02:21.024,2020-01-28 16:07:44.881,24028,7,...,Sentinel-2,393.61 MB,S2A_MSIL2A_20200128T090221_N0213_R007_T33NXC_2...,S2A_OPER_MSI_L1C_TL_SGS__20200128T105959_A0240...,S2A_MSIL2A_20200128T090221_N0213_R007_T33NXC_2...,be23b716-7c71-41d0-9246-cfdd378c9ecb,NaT,NaN,NaN,NaN


In [28]:
products_df

,title,link,link_alternative,link_icon,summary,beginposition,endposition,ingestiondate,orbitnumber,relativeorbitnumber,...,platformname,size,filename,level1cpdiidentifier,identifier,uuid,datatakesensingstart,sensoroperationalmode,tileid,hv_order_tileid
25593eba-df28-4e35-ad6a-e0c7743cd8be,S2A_MSIL2A_20200128T090221_N0213_R007_T33MYT_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2020-01-28T09:02:21.024Z, Instrument: MS...",2020-01-28 09:02:21.024,2020-01-28 09:02:21.024,2020-01-28 16:11:36.895,24028,7,...,Sentinel-2,1016.60 MB,S2A_MSIL2A_20200128T090221_N0213_R007_T33MYT_2...,S2A_OPER_MSI_L1C_TL_SGS__20200128T105959_A0240...,S2A_MSIL2A_20200128T090221_N0213_R007_T33MYT_2...,25593eba-df28-4e35-ad6a-e0c7743cd8be,NaT,NaN,NaN,NaN
2c8ac4fc-e6ea-4871-aca3-0b93a0e5e7a6,S2A_MSIL2A_20200128T090221_N0213_R007_T33MYU_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2020-01-28T09:02:21.024Z, Instrument: MS...",2020-01-28 09:02:21.024,2020-01-28 09:02:21.024,2020-01-28 16:09:01.560,24028,7,...,Sentinel-2,1.01 GB,S2A_MSIL2A_20200128T090221_N0213_R007_T33MYU_2...,S2A_OPER_MSI_L1C_TL_SGS__20200128T105959_A0240...,S2A_MSIL2A_20200128T090221_N0213_R007_T33MYU_2...,2c8ac4fc-e6ea-4871-aca3-0b93a0e5e7a6,NaT,NaN,NaN,NaN
f53e1428-fd5f-4afa-8645-dd239e6b1d70,S2A_MSIL2A_20200128T090221_N0213_R007_T33NYA_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2020-01-28T09:02:21.024Z, Instrument: MS...",2020-01-28 09:02:21.024,2020-01-28 09:02:21.024,2020-01-28 16:08:59.513,24028,7,...,Sentinel-2,1.00 GB,S2A_MSIL2A_20200128T090221_N0213_R007_T33NYA_2...,S2A_OPER_MSI_L1C_TL_SGS__20200128T105959_A0240...,S2A_MSIL2A_20200128T090221_N0213_R007_T33NYA_2...,f53e1428-fd5f-4afa-8645-dd239e6b1d70,NaT,NaN,NaN,NaN
5d840bb8-cd44-467a-b1cd-261f1d8b77f3,S2A_MSIL2A_20200128T090221_N0213_R007_T33NXB_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2020-01-28T09:02:21.024Z, Instrument: MS...",2020-01-28 09:02:21.024,2020-01-28 09:02:21.024,2020-01-28 16:08:14.995,24028,7,...,Sentinel-2,598.48 MB,S2A_MSIL2A_20200128T090221_N0213_R007_T33NXB_2...,S2A_OPER_MSI_L1C_TL_SGS__20200128T105959_A0240...,S2A_MSIL2A_20200128T090221_N0213_R007_T33NXB_2...,5d840bb8-cd44-467a-b1cd-261f1d8b77f3,NaT,NaN,NaN,NaN
be23b716-7c71-41d0-9246-cfdd378c9ecb,S2A_MSIL2A_20200128T090221_N0213_R007_T33NXC_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2020-01-28T09:02:21.024Z, Instrument: MS...",2020-01-28 09:02:21.024,2020-01-28 09:02:21.024,2020-01-28 16:07:44.881,24028,7,...,Sentinel-2,393.61 MB,S2A_MSIL2A_20200128T090221_N0213_R007_T33NXC_2...,S2A_OPER_MSI_L1C_TL_SGS__20200128T105959_A0240...,S2A_MSIL2A_20200128T090221_N0213_R007_T33NXC_2...,be23b716-7c71-41d0-9246-cfdd378c9ecb,NaT,NaN,NaN,NaN
2d0425a9-5973-4ff6-836d-0d09cc99441f,S2A_MSIL2A_20200128T090221_N0213_R007_T34MBC_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2020-01-28T09:02:21.024Z, Instrument: MS...",2020-01-28 09:02:21.024,2020-01-28 09:02:21.024,2020-01-28 16:06:29.147,24028,7,...,Sentinel-2,25.18 MB,S2A_MSIL2A_20200128T090221_N0213_R007_T34MBC_2...,S2A_OPER_MSI_L1C_TL_SGS__20200128T105959_A0240...,S2A_MSIL2A_20200128T090221_N0213_R007_T34MBC_2...,2d0425a9-5973-4ff6-836d-0d09cc99441f,NaT,NaN,NaN,NaN
4914140e-619b-459b-8dac-86cca42a0321,S2A_MSIL2A_20200128T090221_N0213_R007_T33MXU_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/api

In [29]:
api.get_product_odata('5aaeb2ae-947a-4568-ab51-423f651d8f3f', full=True)

{'id': '5aaeb2ae-947a-4568-ab51-423f651d8f3f',
 'title': 'S2A_MSIL1C_20200128T090221_N0208_R007_T33MYS_20200128T105959',
 'size': 692212602,
 'md5': '34426D1F705C314E6E6144EA8A1A116A',
 'date': datetime.datetime(2020, 1, 28, 9, 2, 21, 24000),
 'footprint': 'POLYGON((16.798696514165282 -2.712829405650467,17.785732296130654 -2.710949551849636,17.788420767345347 -3.70313029364641,16.800434009826038 -3.705699752817911,16.798696514165282 -2.712829405650467))',
 'url': "https://scihub.copernicus.eu/apihub/odata/v1/Products('5aaeb2ae-947a-4568-ab51-423f651d8f3f')/$value",
 'Online': True,
 'Creation Date': datetime.datetime(2020, 1, 28, 13, 20, 34, 474000),
 'Ingestion Date': datetime.datetime(2020, 1, 28, 13, 19, 33, 536000),
 'Cloud cover percentage': 2.8988,
 'Datatake sensing start': datetime.datetime(2020, 1, 28, 9, 2, 21, 24000),
 'Date': datetime.datetime(2020, 1, 28, 9, 2, 21, 24000),
 'Degraded ancillary data percentage': 0.0,
 'Degraded MSI data percentage': 0,
 'Filename': 'S2A_MSI

In [30]:
help(api.query)

Help on method query in module sentinelsat.sentinel:

query(area=None, date=None, raw=None, area_relation='Intersects', order_by=None, limit=None, offset=0, **keywords) method of sentinelsat.sentinel.SentinelAPI instance
    Query the OpenSearch API with the coordinates of an area, a date interval
    and any other search keywords accepted by the API.
    
    Parameters
    ----------
    area : str, optional
        The area of interest formatted as a Well-Known Text string.
    date : tuple of (str or datetime) or str, optional
        A time interval filter based on the Sensing Start Time of the products.
        Expects a tuple of (start, end), e.g. ("NOW-1DAY", "NOW").
        The timestamps can be either a Python datetime or a string in one of the
        following formats:
    
            - yyyyMMdd
            - yyyy-MM-ddThh:mm:ss.SSSZ (ISO-8601)
            - yyyy-MM-ddThh:mm:ssZ
            - NOW
            - NOW-<n>DAY(S) (or HOUR(S), MONTH(S), etc.)
            - NOW+<n

In [39]:
products = api.query(footprint,
                     date=(date(2020,1,1), date(2020,1,31)),
                     area_relation='IsWithin',
                     platformname='Sentinel-2',
                     cloudcoverpercentage=(1,5),
                     producttype='S2MSI2A')
    
products_df = api.to_dataframe(products)

products_df

,title,link,link_alternative,link_icon,summary,beginposition,endposition,ingestiondate,orbitnumber,relativeorbitnumber,...,platformserialidentifier,processingbaseline,processinglevel,producttype,platformname,size,filename,level1cpdiidentifier,identifier,uuid
25593eba-df28-4e35-ad6a-e0c7743cd8be,S2A_MSIL2A_20200128T090221_N0213_R007_T33MYT_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2020-01-28T09:02:21.024Z, Instrument: MS...",2020-01-28 09:02:21.024,2020-01-28 09:02:21.024,2020-01-28 16:11:36.895,24028,7,...,Sentinel-2A,02.13,Level-2A,S2MSI2A,Sentinel-2,1016.60 MB,S2A_MSIL2A_20200128T090221_N0213_R007_T33MYT_2...,S2A_OPER_MSI_L1C_TL_SGS__20200128T105959_A0240...,S2A_MSIL2A_20200128T090221_N0213_R007_T33MYT_2...,25593eba-df28-4e35-ad6a-e0c7743cd8be
2c8ac4fc-e6ea-4871-aca3-0b93a0e5e7a6,S2A_MSIL2A_20200128T090221_N0213_R007_T33MYU_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2020-01-28T09:02:21.024Z, Instrument: MS...",2020-01-28 09:02:21.024,2020-01-28 09:02:21.024,2020-01-28 16:09:01.560,24028,7,...,Sentinel-2A,02.13,Level-2A,S2MSI2A,Sentinel-2,1.01 GB,S2A_MSIL2A_20200128T090221_N0213_R007_T33MYU_2...,S2A_OPER_MSI_L1C_TL_SGS__20200128T105959_A0240...,S2A_MSIL2A_20200128T090221_N0213_R007_T33MYU_2...,2c8ac4fc-e6ea-4871-aca3-0b93a0e5e7a6
f53e1428-fd5f-4afa-8645-dd239e6b1d70,S2A_MSIL2A_20200128T090221_N0213_R007_T33NYA_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2020-01-28T09:02:21.024Z, Instrument: MS...",2020-01-28 09:02:21.024,2020-01-28 09:02:21.024,2020-01-28 16:08:59.513,24028,7,...,Sentinel-2A,02.13,Level-2A,S2MSI2A,Sentinel-2,1.00 GB,S2A_MSIL2A_20200128T090221_N0213_R007_T33NYA_2...,S2A_OPER_MSI_L1C_TL_SGS__20200128T105959_A0240...,S2A_MSIL2A_20200128T090221_N0213_R007_T33NYA_2...,f53e1428-fd5f-4afa-8645-dd239e6b1d70
5d840bb8-cd44-467a-b1cd-261f1d8b77f3,S2A_MSIL2A_20200128T090221_N0213_R007_T33NXB_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2020-01-28T09:02:21.024Z, Instrument: MS...",2020-01-28 09:02:21.024,2020-01-28 09:02:21.024,2020-01-28 16:08:14.995,24028,7,...,Sentinel-2A,02.13,Level-2A,S2MSI2A,Sentinel-2,598.48 MB,S2A_MSIL2A_20200128T090221_N0213_R007_T33NXB_2...,S2A_OPER_MSI_L1C_TL_SGS__20200128T105959_A0240...,S2A_MSIL2A_20200128T090221_N0213_R007_T33NXB_2...,5d840bb8-cd44-467a-b1cd-261f1d8b77f3
be23b716-7c71-41d0-9246-cfdd378c9ecb,S2A_MSIL2A_20200128T090221_N0213_R007_T33NXC_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2020-01-28T09:02:21.024Z, Instrument: MS...",2020-01-28 09:02:21.024,2020-01-28 09:02:21.024,2020-01-28 16:07:44.881,24028,7,...,Sentinel-2A,02.13,Level-2A,S2MSI2A,Sentinel-2,393.61 MB,S2A_MSIL2A_20200128T090221_N0213_R007_T33NXC_2...,S2A_OPER_MSI_L1C_TL_SGS__20200128T105959_A0240...,S2A_MSIL2A_20200128T090221_N0213_R007_T33NXC_2...,be23b716-7c71-41d0-9246-cfdd378c9ecb
2d0425a9-5973-4ff6-836d-0d09cc99441f,S2A_MSIL2A_20200128T090221_N0213_R007_T34MBC_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2020-01-28T09:02:21.024Z, Instrument: MS...",2020-01-28 09:02:21.024,2020-01-28 09:02:21.024,2020-01-28 16:06:29.147,24028,7,...,Sentinel-2A,02.13,Level-2A,S2MSI2A,Sentinel-2,25.18 MB,S2A_MSIL2A_20200128T090221_N0213_R007_T34MBC_2...,S2A_OPER_MSI_L1C_TL_SGS__20200128T105959_A0240...,S2A_MSIL2A_20200128T090221_N0213_R007_T34MBC_2...,2d0425a9-5973-4ff6-836d-0d09cc99441f
4914140e-619b-459b-8dac-86cca42a0321,S2A_MSIL2A_2

In [40]:
products_df.columns

Index(['title', 'link', 'link_alternative', 'link_icon', 'summary',
       'beginposition', 'endposition', 'ingestiondate', 'orbitnumber',
       'relativeorbitnumber', 'cloudcoverpercentage',
       'highprobacloudspercentage', 'mediumprobacloudspercentage',
       'snowicepercentage', 'vegetationpercentage', 'waterpercentage',
       'notvegetatedpercentage', 'unclassifiedpercentage', 'gmlfootprint',
       'format', 'instrumentshortname', 'instrumentname', 'footprint',
       's2datatakeid', 'platformidentifier', 'orbitdirection',
       'platformserialidentifier', 'processingbaseline', 'processinglevel',
       'producttype', 'platformname', 'size', 'filename',
       'level1cpdiidentifier', 'identifier', 'uuid'],
      dtype='object')

In [41]:
products = api.query(footprint,
                     date=(date(2020,1,1), date(2020,1,31)),
                     area_relation='IsWithin',
                     platformname='Sentinel-2',
                     cloudcoverpercentage=(1,5),
                     raw='S2MSI2A')

products_df = api.to_dataframe(products)

products_df

,title,link,link_alternative,link_icon,summary,beginposition,endposition,ingestiondate,orbitnumber,relativeorbitnumber,...,platformserialidentifier,processingbaseline,processinglevel,producttype,platformname,size,filename,level1cpdiidentifier,identifier,uuid
25593eba-df28-4e35-ad6a-e0c7743cd8be,S2A_MSIL2A_20200128T090221_N0213_R007_T33MYT_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2020-01-28T09:02:21.024Z, Instrument: MS...",2020-01-28 09:02:21.024,2020-01-28 09:02:21.024,2020-01-28 16:11:36.895,24028,7,...,Sentinel-2A,02.13,Level-2A,S2MSI2A,Sentinel-2,1016.60 MB,S2A_MSIL2A_20200128T090221_N0213_R007_T33MYT_2...,S2A_OPER_MSI_L1C_TL_SGS__20200128T105959_A0240...,S2A_MSIL2A_20200128T090221_N0213_R007_T33MYT_2...,25593eba-df28-4e35-ad6a-e0c7743cd8be
2c8ac4fc-e6ea-4871-aca3-0b93a0e5e7a6,S2A_MSIL2A_20200128T090221_N0213_R007_T33MYU_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2020-01-28T09:02:21.024Z, Instrument: MS...",2020-01-28 09:02:21.024,2020-01-28 09:02:21.024,2020-01-28 16:09:01.560,24028,7,...,Sentinel-2A,02.13,Level-2A,S2MSI2A,Sentinel-2,1.01 GB,S2A_MSIL2A_20200128T090221_N0213_R007_T33MYU_2...,S2A_OPER_MSI_L1C_TL_SGS__20200128T105959_A0240...,S2A_MSIL2A_20200128T090221_N0213_R007_T33MYU_2...,2c8ac4fc-e6ea-4871-aca3-0b93a0e5e7a6
f53e1428-fd5f-4afa-8645-dd239e6b1d70,S2A_MSIL2A_20200128T090221_N0213_R007_T33NYA_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2020-01-28T09:02:21.024Z, Instrument: MS...",2020-01-28 09:02:21.024,2020-01-28 09:02:21.024,2020-01-28 16:08:59.513,24028,7,...,Sentinel-2A,02.13,Level-2A,S2MSI2A,Sentinel-2,1.00 GB,S2A_MSIL2A_20200128T090221_N0213_R007_T33NYA_2...,S2A_OPER_MSI_L1C_TL_SGS__20200128T105959_A0240...,S2A_MSIL2A_20200128T090221_N0213_R007_T33NYA_2...,f53e1428-fd5f-4afa-8645-dd239e6b1d70
5d840bb8-cd44-467a-b1cd-261f1d8b77f3,S2A_MSIL2A_20200128T090221_N0213_R007_T33NXB_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2020-01-28T09:02:21.024Z, Instrument: MS...",2020-01-28 09:02:21.024,2020-01-28 09:02:21.024,2020-01-28 16:08:14.995,24028,7,...,Sentinel-2A,02.13,Level-2A,S2MSI2A,Sentinel-2,598.48 MB,S2A_MSIL2A_20200128T090221_N0213_R007_T33NXB_2...,S2A_OPER_MSI_L1C_TL_SGS__20200128T105959_A0240...,S2A_MSIL2A_20200128T090221_N0213_R007_T33NXB_2...,5d840bb8-cd44-467a-b1cd-261f1d8b77f3
be23b716-7c71-41d0-9246-cfdd378c9ecb,S2A_MSIL2A_20200128T090221_N0213_R007_T33NXC_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2020-01-28T09:02:21.024Z, Instrument: MS...",2020-01-28 09:02:21.024,2020-01-28 09:02:21.024,2020-01-28 16:07:44.881,24028,7,...,Sentinel-2A,02.13,Level-2A,S2MSI2A,Sentinel-2,393.61 MB,S2A_MSIL2A_20200128T090221_N0213_R007_T33NXC_2...,S2A_OPER_MSI_L1C_TL_SGS__20200128T105959_A0240...,S2A_MSIL2A_20200128T090221_N0213_R007_T33NXC_2...,be23b716-7c71-41d0-9246-cfdd378c9ecb
2d0425a9-5973-4ff6-836d-0d09cc99441f,S2A_MSIL2A_20200128T090221_N0213_R007_T34MBC_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2020-01-28T09:02:21.024Z, Instrument: MS...",2020-01-28 09:02:21.024,2020-01-28 09:02:21.024,2020-01-28 16:06:29.147,24028,7,...,Sentinel-2A,02.13,Level-2A,S2MSI2A,Sentinel-2,25.18 MB,S2A_MSIL2A_20200128T090221_N0213_R007_T34MBC_2...,S2A_OPER_MSI_L1C_TL_SGS__20200128T105959_A0240...,S2A_MSIL2A_20200128T090221_N0213_R007_T34MBC_2...,2d0425a9-5973-4ff6-836d-0d09cc99441f
4914140e-619b-459b-8dac-86cca42a0321,S2A_MSIL2A_2

In [42]:
products_df.columns

Index(['title', 'link', 'link_alternative', 'link_icon', 'summary',
       'beginposition', 'endposition', 'ingestiondate', 'orbitnumber',
       'relativeorbitnumber', 'cloudcoverpercentage',
       'highprobacloudspercentage', 'mediumprobacloudspercentage',
       'snowicepercentage', 'vegetationpercentage', 'waterpercentage',
       'notvegetatedpercentage', 'unclassifiedpercentage', 'gmlfootprint',
       'format', 'instrumentshortname', 'instrumentname', 'footprint',
       's2datatakeid', 'platformidentifier', 'orbitdirection',
       'platformserialidentifier', 'processingbaseline', 'processinglevel',
       'producttype', 'platformname', 'size', 'filename',
       'level1cpdiidentifier', 'identifier', 'uuid'],
      dtype='object')

In [43]:
products = api.query(footprint,
                     date=(date(2020,1,1), date(2020,1,31)),
                     area_relation='IsWithin',
                     platformname='Sentinel-2',
                     cloudcoverpercentage=(1,5),
                     raw='1C')

products_df = api.to_dataframe(products)

products_df

,title,link,link_alternative,link_icon,summary,datatakesensingstart,beginposition,endposition,ingestiondate,orbitnumber,...,size,s2datatakeid,producttype,platformidentifier,orbitdirection,platformserialidentifier,processinglevel,identifier,level1cpdiidentifier,uuid
5aaeb2ae-947a-4568-ab51-423f651d8f3f,S2A_MSIL1C_20200128T090221_N0208_R007_T33MYS_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2020-01-28T09:02:21.024Z, Instrument: MS...",2020-01-28 09:02:21.024,2020-01-28 09:02:21.024,2020-01-28 09:02:21.024,2020-01-28 13:19:33.536,24028,...,660.10 MB,GS2A_20200128T090221_024028_N02.08,S2MSI1C,2015-028A,DESCENDING,Sentinel-2A,Level-1C,S2A_MSIL1C_20200128T090221_N0208_R007_T33MYS_2...,S2A_OPER_MSI_L1C_TL_SGS__20200128T105959_A0240...,5aaeb2ae-947a-4568-ab51-423f651d8f3f
baeb894f-f7b7-4807-956e-63dd59da9caa,S2A_MSIL1C_20200128T090221_N0208_R007_T33MXS_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2020-01-28T09:02:21.024Z, Instrument: MS...",2020-01-28 09:02:21.024,2020-01-28 09:02:21.024,2020-01-28 09:02:21.024,2020-01-28 13:17:38.007,24028,...,678.47 MB,GS2A_20200128T090221_024028_N02.08,S2MSI1C,2015-028A,DESCENDING,Sentinel-2A,Level-1C,S2A_MSIL1C_20200128T090221_N0208_R007_T33MXS_2...,S2A_OPER_MSI_L1C_TL_SGS__20200128T105959_A0240...,baeb894f-f7b7-4807-956e-63dd59da9caa
c41c1fe6-ef82-45f8-b54f-210b1be9f081,S2A_MSIL1C_20200127T093241_N0208_R136_T32NRH_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2020-01-27T09:32:41.024Z, Instrument: MS...",2020-01-27 09:32:41.024,2020-01-27 09:32:41.024,2020-01-27 09:32:41.024,2020-01-27 13:31:39.046,24014,...,32.28 MB,GS2A_20200127T093241_024014_N02.08,S2MSI1C,2015-028A,DESCENDING,Sentinel-2A,Level-1C,S2A_MSIL1C_20200127T093241_N0208_R136_T32NRH_2...,S2A_OPER_MSI_L1C_TL_EPAE_20200127T104504_A0240...,c41c1fe6-ef82-45f8-b54f-210b1be9f081
a0d39f12-dc06-4f19-a028-f2fa3184dca7,S2B_MSIL1C_20200126T091139_N0208_R050_T33NWD_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2020-01-26T09:11:39.024Z, Instrument: MS...",2020-01-26 09:11:39.024,2020-01-26 09:11:39.024,2020-01-26 09:11:39.024,2020-01-26 14:29:01.573,15091,...,712.02 MB,GS2B_20200126T091139_015091_N02.08,S2MSI1C,2017-013A,DESCENDING,Sentinel-2B,Level-1C,S2B_MSIL1C_20200126T091139_N0208_R050_T33NWD_2...,S2B_OPER_MSI_L1C_TL_SGS__20200126T111228_A0150...,a0d39f12-dc06-4f19-a028-f2fa3184dca7
cad4df84-7ded-4e36-97bb-b962f7e4674c,S2B_MSIL1C_20200126T091139_N0208_R050_T33MUU_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2020-01-26T09:11:39.024Z, Instrument: MS...",2020-01-26 09:11:39.024,2020-01-26 09:11:39.024,2020-01-26 09:11:39.024,2020-01-26 14:26:24.650,15091,...,639.70 MB,GS2B_20200126T091139_015091_N02.08,S2MSI1C,2017-013A,DESCENDING,Sentinel-2B,Level-1C,S2B_MSIL1C_20200126T091139_N0208_R050_T33MUU_2...,S2B_OPER_MSI_L1C_TL_SGS__20200126T111228_A0150...,cad4df84-7ded-4e36-97bb-b962f7e4674c
a0d73bda-e39e-4883-b7b5-c64fd876111b,S2B_MSIL1C_20200126T091139_N0208_R050_T33MUT_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2020-01-26T09:11:39.024Z, Instrument: MS...",2020-01-26 09:11:39.024,2020-01-26 09:11:39.024,2020-01-26 09:11:39.024,2020-01-26 14:25:05.169,15091,...,728.19 MB,GS2B_20200126T091139_015091_N02.08,S2MSI1C,2017-013A,DESCENDING,Sentinel-2B,Level-1C,S2B_MSIL1C_20200126T091139_N0208_R050_T33MUT_2...,S2B_OPER_MSI_L1C_TL_SGS__20200126T111228_A0150...,a0d73bda-e

In [44]:
products_df.columns

Index(['title', 'link', 'link_alternative', 'link_icon', 'summary',
       'datatakesensingstart', 'beginposition', 'endposition', 'ingestiondate',
       'orbitnumber', 'relativeorbitnumber', 'cloudcoverpercentage',
       'sensoroperationalmode', 'gmlfootprint', 'footprint', 'tileid',
       'hv_order_tileid', 'format', 'processingbaseline', 'platformname',
       'filename', 'instrumentname', 'instrumentshortname', 'size',
       's2datatakeid', 'producttype', 'platformidentifier', 'orbitdirection',
       'platformserialidentifier', 'processinglevel', 'identifier',
       'level1cpdiidentifier', 'uuid'],
      dtype='object')

In [45]:
products

OrderedDict([('5aaeb2ae-947a-4568-ab51-423f651d8f3f',
              {'title': 'S2A_MSIL1C_20200128T090221_N0208_R007_T33MYS_20200128T105959',
               'link': "https://scihub.copernicus.eu/apihub/odata/v1/Products('5aaeb2ae-947a-4568-ab51-423f651d8f3f')/$value",
               'link_alternative': "https://scihub.copernicus.eu/apihub/odata/v1/Products('5aaeb2ae-947a-4568-ab51-423f651d8f3f')/",
               'link_icon': "https://scihub.copernicus.eu/apihub/odata/v1/Products('5aaeb2ae-947a-4568-ab51-423f651d8f3f')/Products('Quicklook')/$value",
               'summary': 'Date: 2020-01-28T09:02:21.024Z, Instrument: MSI, Mode: , Satellite: Sentinel-2, Size: 660.10 MB',
               'datatakesensingstart': datetime.datetime(2020, 1, 28, 9, 2, 21, 24000),
               'beginposition': datetime.datetime(2020, 1, 28, 9, 2, 21, 24000),
               'endposition': datetime.datetime(2020, 1, 28, 9, 2, 21, 24000),
               'ingestiondate': datetime.datetime(2020, 1, 28, 13, 19, 

In [46]:
products = api.query(footprint,
                     date=(date(2020,1,1), date(2020,1,31)),
                     area_relation='IsWithin',
                     platformname='Sentinel-2',
                     cloudcoverpercentage=(1,5),
                     raw='2A')

products_df = api.to_dataframe(products)

products_df

,title,link,link_alternative,link_icon,summary,beginposition,endposition,ingestiondate,orbitnumber,relativeorbitnumber,...,platformname,size,filename,level1cpdiidentifier,identifier,uuid,datatakesensingstart,sensoroperationalmode,tileid,hv_order_tileid
25593eba-df28-4e35-ad6a-e0c7743cd8be,S2A_MSIL2A_20200128T090221_N0213_R007_T33MYT_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2020-01-28T09:02:21.024Z, Instrument: MS...",2020-01-28 09:02:21.024,2020-01-28 09:02:21.024,2020-01-28 16:11:36.895,24028,7,...,Sentinel-2,1016.60 MB,S2A_MSIL2A_20200128T090221_N0213_R007_T33MYT_2...,S2A_OPER_MSI_L1C_TL_SGS__20200128T105959_A0240...,S2A_MSIL2A_20200128T090221_N0213_R007_T33MYT_2...,25593eba-df28-4e35-ad6a-e0c7743cd8be,NaT,NaN,NaN,NaN
2c8ac4fc-e6ea-4871-aca3-0b93a0e5e7a6,S2A_MSIL2A_20200128T090221_N0213_R007_T33MYU_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2020-01-28T09:02:21.024Z, Instrument: MS...",2020-01-28 09:02:21.024,2020-01-28 09:02:21.024,2020-01-28 16:09:01.560,24028,7,...,Sentinel-2,1.01 GB,S2A_MSIL2A_20200128T090221_N0213_R007_T33MYU_2...,S2A_OPER_MSI_L1C_TL_SGS__20200128T105959_A0240...,S2A_MSIL2A_20200128T090221_N0213_R007_T33MYU_2...,2c8ac4fc-e6ea-4871-aca3-0b93a0e5e7a6,NaT,NaN,NaN,NaN
f53e1428-fd5f-4afa-8645-dd239e6b1d70,S2A_MSIL2A_20200128T090221_N0213_R007_T33NYA_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2020-01-28T09:02:21.024Z, Instrument: MS...",2020-01-28 09:02:21.024,2020-01-28 09:02:21.024,2020-01-28 16:08:59.513,24028,7,...,Sentinel-2,1.00 GB,S2A_MSIL2A_20200128T090221_N0213_R007_T33NYA_2...,S2A_OPER_MSI_L1C_TL_SGS__20200128T105959_A0240...,S2A_MSIL2A_20200128T090221_N0213_R007_T33NYA_2...,f53e1428-fd5f-4afa-8645-dd239e6b1d70,NaT,NaN,NaN,NaN
5d840bb8-cd44-467a-b1cd-261f1d8b77f3,S2A_MSIL2A_20200128T090221_N0213_R007_T33NXB_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2020-01-28T09:02:21.024Z, Instrument: MS...",2020-01-28 09:02:21.024,2020-01-28 09:02:21.024,2020-01-28 16:08:14.995,24028,7,...,Sentinel-2,598.48 MB,S2A_MSIL2A_20200128T090221_N0213_R007_T33NXB_2...,S2A_OPER_MSI_L1C_TL_SGS__20200128T105959_A0240...,S2A_MSIL2A_20200128T090221_N0213_R007_T33NXB_2...,5d840bb8-cd44-467a-b1cd-261f1d8b77f3,NaT,NaN,NaN,NaN
be23b716-7c71-41d0-9246-cfdd378c9ecb,S2A_MSIL2A_20200128T090221_N0213_R007_T33NXC_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2020-01-28T09:02:21.024Z, Instrument: MS...",2020-01-28 09:02:21.024,2020-01-28 09:02:21.024,2020-01-28 16:07:44.881,24028,7,...,Sentinel-2,393.61 MB,S2A_MSIL2A_20200128T090221_N0213_R007_T33NXC_2...,S2A_OPER_MSI_L1C_TL_SGS__20200128T105959_A0240...,S2A_MSIL2A_20200128T090221_N0213_R007_T33NXC_2...,be23b716-7c71-41d0-9246-cfdd378c9ecb,NaT,NaN,NaN,NaN
2d0425a9-5973-4ff6-836d-0d09cc99441f,S2A_MSIL2A_20200128T090221_N0213_R007_T34MBC_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2020-01-28T09:02:21.024Z, Instrument: MS...",2020-01-28 09:02:21.024,2020-01-28 09:02:21.024,2020-01-28 16:06:29.147,24028,7,...,Sentinel-2,25.18 MB,S2A_MSIL2A_20200128T090221_N0213_R007_T34MBC_2...,S2A_OPER_MSI_L1C_TL_SGS__20200128T105959_A0240...,S2A_MSIL2A_20200128T090221_N0213_R007_T34MBC_2...,2d0425a9-5973-4ff6-836d-0d09cc99441f,NaT,NaN,NaN,NaN
4914140e-619b-459b-8dac-86cca42a0321,S2A_MSIL2A_20200128T090221_N0213_R007_T33MXU_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/api

In [47]:
products_df.columns

Index(['title', 'link', 'link_alternative', 'link_icon', 'summary',
       'beginposition', 'endposition', 'ingestiondate', 'orbitnumber',
       'relativeorbitnumber', 'cloudcoverpercentage',
       'highprobacloudspercentage', 'mediumprobacloudspercentage',
       'snowicepercentage', 'vegetationpercentage', 'waterpercentage',
       'notvegetatedpercentage', 'unclassifiedpercentage', 'gmlfootprint',
       'format', 'instrumentshortname', 'instrumentname', 'footprint',
       's2datatakeid', 'platformidentifier', 'orbitdirection',
       'platformserialidentifier', 'processingbaseline', 'processinglevel',
       'producttype', 'platformname', 'size', 'filename',
       'level1cpdiidentifier', 'identifier', 'uuid', 'datatakesensingstart',
       'sensoroperationalmode', 'tileid', 'hv_order_tileid'],
      dtype='object')

In [48]:
products_df.isnull().sum()

title                           0
link                            0
link_alternative                0
link_icon                       0
summary                         0
beginposition                   0
endposition                     0
ingestiondate                   0
orbitnumber                     0
relativeorbitnumber             0
cloudcoverpercentage            0
highprobacloudspercentage      15
mediumprobacloudspercentage    15
snowicepercentage              15
vegetationpercentage           15
waterpercentage                15
notvegetatedpercentage         15
unclassifiedpercentage         15
gmlfootprint                    0
format                          0
instrumentshortname             0
instrumentname                  0
footprint                       0
s2datatakeid                    0
platformidentifier              0
orbitdirection                  0
platformserialidentifier        0
processingbaseline              0
processinglevel                 0
producttype   

In [49]:
df2 = products_df[products_df['title'].str.contains('L2A')]

print(df2.shape)

df2.isnull().sum()

(38, 40)


title                           0
link                            0
link_alternative                0
link_icon                       0
summary                         0
beginposition                   0
endposition                     0
ingestiondate                   0
orbitnumber                     0
relativeorbitnumber             0
cloudcoverpercentage            0
highprobacloudspercentage       0
mediumprobacloudspercentage     0
snowicepercentage               0
vegetationpercentage            0
waterpercentage                 0
notvegetatedpercentage          0
unclassifiedpercentage          0
gmlfootprint                    0
format                          0
instrumentshortname             0
instrumentname                  0
footprint                       0
s2datatakeid                    0
platformidentifier              0
orbitdirection                  0
platformserialidentifier        0
processingbaseline              0
processinglevel                 0
producttype   